In [6]:
import pandas as pd
from pathlib import Path

# Directory containing per-object CSVs
DATA_DIR = Path("agnmass_dump/reverb_tables")

# Collect all CSV files
csv_files = sorted(DATA_DIR.glob("details_varname_*_reverb_with_meta.csv"))

print(f"Found {len(csv_files)} CSV files")

# Read and concatenate
dfs = []
for f in csv_files:
    df = pd.read_csv(f, sep=";")
    dfs.append(df)

merged_df = pd.concat(dfs, ignore_index=True)

# Save merged CSV
OUTPUT_PATH = DATA_DIR / "qc_project_merged_data.csv"
merged_df.to_csv(OUTPUT_PATH, sep=";", index=False, encoding="utf-8-sig")

print(f"Merged CSV saved to: {OUTPUT_PATH}")
print(f"Total number of rows in merged dataset: {merged_df.shape[0]}")


Found 95 CSV files
Merged CSV saved to: agnmass_dump/reverb_tables/qc_project_merged_data.csv
Total number of rows in merged dataset: 299


/tmp/ipykernel_199151/1414516086.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_df = pd.concat(dfs, ignore_index=True)


In [7]:
import numpy as np

# Replace common missing-value markers with NaN
missing_markers = [
    "...", "…", "", " ", "None", "none", "NULL", "null"
]

merged_df_clean = merged_df.replace(missing_markers, np.nan)

# Optional: show how many NaNs per column (sanity check)
na_summary = merged_df_clean.isna().sum().sort_values(ascending=False)
na_summary.head(10)


/tmp/ipykernel_199151/991415248.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df_clean = merged_df.replace(missing_markers, np.nan)


mbh_rm_modeling             196
mbh_rm_ref                  196
alternate_names             171
τJAV Ref.                   147
τJAV (days)                 147
log LAGN,5100 (ergs s-1)    143
L Ref.                      143
τpeak (days)                 71
σline (km s-1)               70
FWHM (km s-1)                67
dtype: int64

In [6]:
# # Choose identifier (varname is safest)
# ID_COL = "varname"

# total_objects = merged_df_clean[ID_COL].nunique()
# print(f"Total unique objects: {total_objects}\n")

# def report_missing(label_col):
#     """
#     Reports how many unique objects are missing a given label.
#     """
#     # For each object, check if ALL rows are NaN for that label
#     missing_by_object = (
#         merged_df_clean
#         .groupby(ID_COL)[label_col]
#         .apply(lambda s: s.isna().all())
#     )

#     num_missing = missing_by_object.sum()
#     percent_missing = 100 * num_missing / total_objects

#     print(f"{label_col}:")
#     print(f"  Objects missing value : {num_missing}")
#     print(f"  Percentage missing    : {percent_missing:.2f}%")
#     print(f"  Objects with value    : {total_objects - num_missing}")
#     print("-" * 50)

# # Run reports
# report_missing("mbh_hbeta_only")
# report_missing("mbh_all_lines")
# report_missing("mbh_rm_modeling")


In [8]:
import re
import numpy as np
import pandas as pd
from pathlib import Path

# ---------- Load merged CSV ----------
path = Path("agnmass_dump/reverb_tables/qc_project_merged_data.csv")

# Your files are usually ';' delimited (German Excel-friendly). This tries ';' first, then ','.
try:
    df = pd.read_csv(path, sep=";", encoding="utf-8-sig")
except Exception:
    df = pd.read_csv(path, sep=",", encoding="utf-8-sig")

print("Loaded:", path)
print("Shape:", df.shape)

# ---------- Helpers for robust missing detection ----------
# Matches ellipsis forms like:
#   "..." , "…", ". . .", ".  .   .", "... (10 ^7 M_sun)", "… something"
ELLIPSIS_START_RE = r"^\s*(?:\.\s*){3,}.*$|^\s*….*$"
ELLIPSIS_ONLY_RE  = r"^\s*(?:\.\s*){3,}\s*$|^\s*…\s*$"

# ---------- (1) Global missing cleanup ----------
# 1A) Whitespace-only -> NaN
df = df.replace(to_replace=r"^\s*$", value=np.nan, regex=True)

# 1B) Ellipsis-only cells -> NaN
df = df.replace(to_replace=ELLIPSIS_ONLY_RE, value=np.nan, regex=True)

# 1C) (Optional global) cells starting with ellipsis -> NaN
# This is safe for this dataset and catches embedded missing like "... (10^7 ...)"
df = df.replace(to_replace=ELLIPSIS_START_RE, value=np.nan, regex=True)

# 1D) Strip string cells (helps matching + cleaner viewing)
for c in df.columns:
    if df[c].dtype == "object":
        df[c] = df[c].apply(lambda x: x.strip() if isinstance(x, str) else x)

# ---------- (2) Generic uncertainty parser (tau/sigma/FWHM/logL etc.) ----------
def parse_value_and_uncertainty(x):
    """
    Returns (val, plus, minus) as floats from strings like:
      '0.74 (+0.49/-0.49)'
      '3001 (+/- 277)'
      '43.70 (+/- 0.06)'
    If only a number exists: (val, nan, nan)
    If missing/unparseable: (nan, nan, nan)
    """
    if pd.isna(x):
        return (np.nan, np.nan, np.nan)

    s = str(x)
    s = s.replace("\n", " ")
    s = re.sub(r"\s+", " ", s).strip()

    # If it still begins with ellipsis for any reason, treat as missing
    if re.match(ELLIPSIS_START_RE, s):
        return (np.nan, np.nan, np.nan)

    # First number = central value
    m0 = re.search(r"([-+]?\d+(?:\.\d+)?)", s)
    if not m0:
        return (np.nan, np.nan, np.nan)
    val = float(m0.group(1))

    # Asymmetric: (+a/-b)
    m = re.search(r"\(\s*\+\s*([\d.]+)\s*/\s*-\s*([\d.]+)\s*\)", s)
    if m:
        return (val, float(m.group(1)), float(m.group(2)))

    # Symmetric: (+/- a)
    m = re.search(r"\(\s*\+/-\s*([\d.]+)\s*\)", s)
    if m:
        a = float(m.group(1))
        return (val, a, a)

    return (val, np.nan, np.nan)

def add_uncertainty_columns(df, col):
    vals = df[col].apply(
        lambda x: pd.Series(
            parse_value_and_uncertainty(x),
            index=[f"{col}_val", f"{col}_plus", f"{col}_minus"]
        )
    )
    return pd.concat([df, vals], axis=1)

# Measurement columns to parse (avoid refs)
mbh_cols = ["mbh_hbeta_only", "mbh_all_lines", "mbh_rm_modeling"]
candidate_cols = []

for c in df.columns:
    if c in mbh_cols:
        continue

    cname = str(c)
    cname_lower = cname.lower()

    # skip reference columns
    if "ref" in cname_lower:
        continue

    # typical measurement columns contain these substrings
    if any(k in cname for k in ["τ", "sigma", "σ", "FWHM", "log L", "logL", "L_AGN", "LAGN"]):
        candidate_cols.append(c)

for c in candidate_cols:
    df = add_uncertainty_columns(df, c)

print(f"Added uncertainty numeric columns for {len(candidate_cols)} measurement columns.")

# ---------- (3) MBH parser: mantissa + uncertainty + exponent + log10 mass ----------
def parse_mbh_string(x):
    """
    Parses strings like:
      '1.961 (^+ 0.516 / _- 0.392) (10 ^7 M _sun)'
      '1.699 (+ 0.171/- 0.164) (10^7 M_sun)'

    Returns:
      mantissa, plus, minus, exponent
    """
    if pd.isna(x):
        return (np.nan, np.nan, np.nan, np.nan)

    s = str(x)
    s = s.replace("\n", " ")
    s = re.sub(r"\s+", " ", s).strip()

    # CRITICAL: if MBH string starts with ellipsis, it is missing (even if it contains "10^7")
    if re.match(ELLIPSIS_START_RE, s):
        return (np.nan, np.nan, np.nan, np.nan)

    # Central mantissa = first float
    m0 = re.search(r"([-+]?\d+(?:\.\d+)?)", s)
    if not m0:
        return (np.nan, np.nan, np.nan, np.nan)
    mantissa = float(m0.group(1))

    # Normalize sup/sub style artifacts
    norm = s.replace("^+", "+").replace("_-", "-")
    norm = norm.replace("(^", "(").replace("/ _", "/ -").replace("( ^", "(")

    plus = minus = np.nan

    m = re.search(r"\(\s*\+\s*([\d.]+)\s*/\s*-\s*([\d.]+)\s*\)", norm)
    if m:
        plus, minus = float(m.group(1)), float(m.group(2))
    else:
        m = re.search(r"\(\s*\+/-\s*([\d.]+)\s*\)", norm)
        if m:
            plus = minus = float(m.group(1))

    # Exponent from '10 ^7' or '10^7'
    exp = np.nan
    mexp = re.search(r"10\s*\^?\s*([+-]?\d+)", norm)
    if mexp:
        exp = float(mexp.group(1))

    return (mantissa, plus, minus, exp)

def add_mbh_columns(df, col):
    # extra pre-clean: convert any ellipsis-starting strings to NaN in MBH columns
    df[col] = df[col].replace(to_replace=ELLIPSIS_START_RE, value=np.nan, regex=True)

    out = df[col].apply(
        lambda x: pd.Series(
            parse_mbh_string(x),
            index=[f"{col}_mantissa", f"{col}_plus_mant", f"{col}_minus_mant", f"{col}_exp"]
        )
    )
    df = pd.concat([df, out], axis=1)

    mant = df[f"{col}_mantissa"]
    exp = df[f"{col}_exp"]
    plus = df[f"{col}_plus_mant"]
    minus = df[f"{col}_minus_mant"]

    # Only compute when exp is present
    factor = 10 ** exp

    df[f"{col}_msun_val"] = mant * factor
    df[f"{col}_msun_plus"] = (mant + plus) * factor
    df[f"{col}_msun_minus"] = (mant - minus) * factor

    # log10(M/Msun)
    df[f"{col}_log10_val"] = np.log10(mant) + exp
    df[f"{col}_log10_plus"] = np.log10((mant + plus) * factor) - df[f"{col}_log10_val"]
    df[f"{col}_log10_minus"] = df[f"{col}_log10_val"] - np.log10((mant - minus) * factor)

    return df

for col in mbh_cols:
    if col in df.columns:
        df = add_mbh_columns(df, col)

print("Added MBH parsed columns (mantissa/errors/exponent/msun/log10) for:", [c for c in mbh_cols if c in df.columns])

# ---------- (4) Parse JD Range ----------
jd_cols = [c for c in df.columns if "JD Range" in str(c)]
if jd_cols:
    jd_col = jd_cols[0]

    def parse_jd_range(x):
        if pd.isna(x):
            return (np.nan, np.nan, np.nan, np.nan)
        s = str(x).strip()
        m = re.search(r"(\d+(?:\.\d+)?)\s*-\s*(\d+(?:\.\d+)?)", s)
        if not m:
            return (np.nan, np.nan, np.nan, np.nan)
        a, b = float(m.group(1)), float(m.group(2))
        return (a, b, b - a, 0.5 * (a + b))

    jd_out = df[jd_col].apply(
        lambda x: pd.Series(parse_jd_range(x), index=["jd_start", "jd_end", "jd_span", "jd_mid"])
    )
    df = pd.concat([df, jd_out], axis=1)
    print("Parsed JD range into jd_start/jd_end/jd_span/jd_mid using:", jd_col)

# ---------- Final ----------
merged_df_clean = df

print("\nDone. Cleaned + feature-augmented dataframe:")
print("Shape:", merged_df_clean.shape)

# ---------- Save cleaned & augmented data ----------
OUT_PATH = Path("agnmass_dump/reverb_tables/qc_project_cleaned_data.csv")
merged_df_clean.to_csv(OUT_PATH, sep=";", index=False, encoding="utf-8-sig")
print(f"Cleaned data saved to: {OUT_PATH}")


Loaded: agnmass_dump/reverb_tables/qc_project_merged_data.csv
Shape: (299, 25)
Added uncertainty numeric columns for 6 measurement columns.
Added MBH parsed columns (mantissa/errors/exponent/msun/log10) for: ['mbh_hbeta_only', 'mbh_all_lines', 'mbh_rm_modeling']
Parsed JD range into jd_start/jd_end/jd_span/jd_mid using: JD Range (days)

Done. Cleaned + feature-augmented dataframe:
Shape: (299, 77)
Cleaned data saved to: agnmass_dump/reverb_tables/qc_project_cleaned_data.csv


In [10]:
# read agnmass_dump/reverb_tables/qc_project_cleaned_data.csv and print the name of all its columns
import pandas as pd
from pathlib import Path

path = Path("agnmass_dump/reverb_tables/qc_project_cleaned_data.csv")
df = pd.read_csv(path, sep=";", encoding="utf-8-sig")
print("Columns in the cleaned data:")
print(df.columns)

Columns in the cleaned data:
Index(['Line', 'JD Range (days)', 'τJAV (days)', 'τJAV Ref.', 'τcent (days)',
       'τpeak (days)', 'σline (km s-1)', 'FWHM (km s-1)', 'RM Ref(s)',
       'log LAGN,5100 (ergs s-1)', 'L Ref.', 'varname', 'object_name',
       'alternate_names', 'ra', 'dec', 'z', 'dl_mpc', 'da_mpc', 'f_used',
       'mbh_hbeta_only', 'mbh_all_lines', 'mbh_rm_modeling', 'mbh_rm_ref',
       'source_url', 'τJAV (days)_val', 'τJAV (days)_plus',
       'τJAV (days)_minus', 'τcent (days)_val', 'τcent (days)_plus',
       'τcent (days)_minus', 'τpeak (days)_val', 'τpeak (days)_plus',
       'τpeak (days)_minus', 'σline (km s-1)_val', 'σline (km s-1)_plus',
       'σline (km s-1)_minus', 'FWHM (km s-1)_val', 'FWHM (km s-1)_plus',
       'FWHM (km s-1)_minus', 'log LAGN,5100 (ergs s-1)_val',
       'log LAGN,5100 (ergs s-1)_plus', 'log LAGN,5100 (ergs s-1)_minus',
       'mbh_hbeta_only_mantissa', 'mbh_hbeta_only_plus_mant',
       'mbh_hbeta_only_minus_mant', 'mbh_hbeta_only_exp',

In [9]:
import pandas as pd
from pathlib import Path

# Load the cleaned & augmented dataset
CLEAN_PATH = Path("agnmass_dump/reverb_tables/qc_project_cleaned_data.csv")

# Your cleaned CSV was saved with sep=";" and encoding="utf-8-sig"
merged_df_clean = pd.read_csv(CLEAN_PATH, sep=";", encoding="utf-8-sig")

print("Loaded:", CLEAN_PATH)
print("Shape:", merged_df_clean.shape)

# Choose identifier (varname is safest)
ID_COL = "varname"

total_objects = merged_df_clean[ID_COL].nunique()
print(f"\nTotal unique objects: {total_objects}\n")

def report_missing_object_level(label_col: str):
    """
    Reports how many unique objects are missing a given label column.
    Missing for an object = all rows for that object are NaN for that column.
    """
    missing_by_object = (
        merged_df_clean
        .groupby(ID_COL)[label_col]
        .apply(lambda s: s.isna().all())
    )

    num_missing = int(missing_by_object.sum())
    percent_missing = 100 * num_missing / total_objects

    print(f"{label_col}:")
    print(f"  Objects missing value : {num_missing}")
    print(f"  Percentage missing    : {percent_missing:.2f}%")
    print(f"  Objects with value    : {total_objects - num_missing}")
    print("-" * 60)

# Run reports on the NEW numeric targets
report_missing_object_level("mbh_hbeta_only_log10_val")
report_missing_object_level("mbh_all_lines_log10_val")
report_missing_object_level("mbh_rm_modeling_log10_val")


Loaded: agnmass_dump/reverb_tables/qc_project_cleaned_data.csv
Shape: (299, 77)

Total unique objects: 86

mbh_hbeta_only_log10_val:
  Objects missing value : 19
  Percentage missing    : 22.09%
  Objects with value    : 67
------------------------------------------------------------
mbh_all_lines_log10_val:
  Objects missing value : 16
  Percentage missing    : 18.60%
  Objects with value    : 70
------------------------------------------------------------
mbh_rm_modeling_log10_val:
  Objects missing value : 71
  Percentage missing    : 82.56%
  Objects with value    : 15
------------------------------------------------------------


In [10]:
# import numpy as np
# import pandas as pd
# from pathlib import Path

# # ----------------------------
# # Load cleaned dataset
# # ----------------------------
# IN_PATH = Path("agnmass_dump/reverb_tables/qc_project_cleaned_data.csv")
# df = pd.read_csv(IN_PATH, sep=";", encoding="utf-8-sig")
# print("Loaded:", IN_PATH, "shape:", df.shape)

# # ----------------------------
# # Config / column names
# # ----------------------------
# ID_COL = "varname"          # or "object_name"
# LINE_COL = "Line"

# # Parsed numeric columns already in your cleaned data
# TAU_COL = "τcent (days)_val"
# SIGMA_COL = "σline (km s-1)_val"

# # Existing Hβ-only label columns (parsed)
# HB_LOG_COL = "mbh_hbeta_only_log10_val"
# HB_LOG_PLUS = "mbh_hbeta_only_log10_plus"
# HB_LOG_MINUS = "mbh_hbeta_only_log10_minus"

# # If you want to also keep a Msun version
# HB_MSUN_COL = "mbh_hbeta_only_msun_val"

# # Output columns we will create
# TARGET_LOG_COL = "target_log10_mbh_hbeta"
# TARGET_SRC_COL = "target_source"   # 'website' vs 'computed'
# TARGET_ROWFLAG = "is_hbeta_row"     # row is an Hβ measurement
# TARGET_COMPUTED_FLAG = "is_target_computed"  # True if we computed due to missing website value

# # Constants for computation
# F_DEFAULT = 4.3  # as used by the site default
# LOG10_C = np.log10(2.99792458e8)      # m/s
# LOG10_G = np.log10(6.67430e-11)       # SI
# LOG10_MSUN = np.log10(1.98847e30)     # kg
# LOG10_DAY = np.log10(86400.0)         # s/day
# LOG10_F = np.log10(F_DEFAULT)

# # ----------------------------
# # 1) Build Hβ-only target for all rows (website if present; computed if missing)
# # ----------------------------

# # Identify Hβ rows (the site uses "Hβ", sometimes with extra text like "Hβ λ4861")
# df[TARGET_ROWFLAG] = df[LINE_COL].astype(str).str.contains("Hβ", na=False)

# # Start with website value if it exists
# df[TARGET_LOG_COL] = df[HB_LOG_COL]

# # Compute log10(MBH/Msun) for Hβ rows where website value is missing
# # Formula:
# #   MBH = f * (c * tau_sec) * (V_mps^2) / G
# #   tau_sec = tau_days * 86400
# #   V_mps = sigma_kmps * 1000
# #
# # In log10 space:
# #   log10(MBH_kg) = log10(f) + log10(c) + log10(tau_days) + log10(86400)
# #                  + 2*(log10(sigma_kmps) + 3) - log10(G)
# #   log10(MBH/Msun) = log10(MBH_kg) - log10(Msun)

# mask_need_compute = (
#     df[TARGET_ROWFLAG]
#     & df[TARGET_LOG_COL].isna()
#     & df[TAU_COL].notna()
#     & df[SIGMA_COL].notna()
# )

# # Safe logs
# tau_days = df.loc[mask_need_compute, TAU_COL].astype(float)
# sigma_kmps = df.loc[mask_need_compute, SIGMA_COL].astype(float)

# computed_log10 = (
#     LOG10_F
#     + LOG10_C
#     + np.log10(tau_days)
#     + LOG10_DAY
#     + 2.0 * (np.log10(sigma_kmps) + 3.0)  # km/s -> m/s = +3 in log10
#     - LOG10_G
#     - LOG10_MSUN
# )

# df.loc[mask_need_compute, TARGET_LOG_COL] = computed_log10

# # Flags describing where target came from
# df[TARGET_COMPUTED_FLAG] = False
# df.loc[mask_need_compute, TARGET_COMPUTED_FLAG] = True

# df[TARGET_SRC_COL] = "website"
# df.loc[df[TARGET_COMPUTED_FLAG], TARGET_SRC_COL] = "computed"

# # Optional: you may prefer to set non-Hβ rows' target to NaN (since they aren't Hβ measurements)
# # If you want that, uncomment:
# # df.loc[~df[TARGET_ROWFLAG], TARGET_LOG_COL] = np.nan

# print("\nAfter filling targets:")
# print("  Total rows:", len(df))
# print("  Hβ rows:", int(df[TARGET_ROWFLAG].sum()))
# print("  Computed targets (rows):", int(df[TARGET_COMPUTED_FLAG].sum()))
# print("  Missing target among Hβ rows:", int(df.loc[df[TARGET_ROWFLAG], TARGET_LOG_COL].isna().sum()))

# # ----------------------------
# # 2) Save Version A: raw per-row targets (website or computed)
# # ----------------------------
# OUT_RAW = Path("agnmass_dump/reverb_tables/qc_project_hbeta_target_per_row.csv")
# df.to_csv(OUT_RAW, sep=";", index=False, encoding="utf-8-sig")
# print("\nSaved per-row version to:", OUT_RAW)

# # ----------------------------
# # 3) Version B: average computed targets per object (only for computed ones), then substitute
# # ----------------------------

# df_avg = df.copy()

# # For each object, compute mean of computed targets across its Hβ rows
# # (Only computed ones; website ones remain untouched.)
# computed_means = (
#     df_avg[df_avg[TARGET_COMPUTED_FLAG]]
#     .groupby(ID_COL)[TARGET_LOG_COL]
#     .mean()
# )

# # Substitute: for rows where target is computed, replace with that object's mean
# df_avg.loc[df_avg[TARGET_COMPUTED_FLAG], TARGET_LOG_COL] = df_avg.loc[df_avg[TARGET_COMPUTED_FLAG], ID_COL].map(computed_means)

# # Add a convenience flag indicating we performed averaging (still computed, but aggregated)
# df_avg["is_target_computed_avg"] = False
# df_avg.loc[df_avg[TARGET_COMPUTED_FLAG], "is_target_computed_avg"] = True

# OUT_AVG = Path("agnmass_dump/reverb_tables/qc_project_hbeta_target_computed_avg.csv")
# df_avg.to_csv(OUT_AVG, sep=";", index=False, encoding="utf-8-sig")
# print("Saved averaged-computed version to:", OUT_AVG)


In [11]:
import numpy as np
import pandas as pd
from pathlib import Path

IN_PATH = Path("agnmass_dump/reverb_tables/qc_project_cleaned_data.csv")
df = pd.read_csv(IN_PATH, sep=";", encoding="utf-8-sig")
print("Loaded:", IN_PATH, "shape:", df.shape)

# ----------------------------
# Columns
# ----------------------------
ID_COL = "varname"  # safest object id
LINE_COL = "Line"

TAU_COL = "τcent (days)_val"
SIGMA_COL = "σline (km s-1)_val"

# Website-derived parsed target (object-level metadata repeated on all rows)
HB_WEBSITE_LOG = "mbh_hbeta_only_log10_val"

# ----------------------------
# Constants for virial computation
# MBH = f * (c * tau) * (V^2) / G
# tau in seconds, V in m/s
# target = log10(MBH / Msun)
# ----------------------------
F_DEFAULT = 4.3
C = 2.99792458e8         # m/s
G = 6.67430e-11          # SI
MSUN = 1.98847e30        # kg

# ----------------------------
# 1) Start: object-level target from website if present
# ----------------------------
df["target_hbeta_log10"] = df[HB_WEBSITE_LOG]
df["target_hbeta_source"] = np.where(df["target_hbeta_log10"].notna(), "website", "missing")

# ----------------------------
# 2) For objects missing website Hβ target, try computing it from their Hβ rows
# ----------------------------
is_hbeta_row = df[LINE_COL].astype(str).str.contains("Hβ", na=False)

# Rows usable for computation
usable = is_hbeta_row & df[TAU_COL].notna() & df[SIGMA_COL].notna()

# Compute per-row virial log10(M/Msun) for usable Hβ rows
tau_sec = df.loc[usable, TAU_COL].astype(float) * 86400.0
v_mps = df.loc[usable, SIGMA_COL].astype(float) * 1000.0

mbh_kg = F_DEFAULT * (C * tau_sec) * (v_mps ** 2) / G
row_log10 = np.log10(mbh_kg / MSUN)

# Store the per-row computed values (only for Hβ rows where we can compute)
df["hbeta_row_computed_log10"] = np.nan
df.loc[usable, "hbeta_row_computed_log10"] = row_log10

# Compute an object-level mean from available computed Hβ rows
computed_obj_mean = (
    df.loc[usable]
      .groupby(ID_COL)["hbeta_row_computed_log10"]
      .mean()
)

# Fill missing target_hbeta_log10 at the OBJECT level using that mean
missing_obj = df["target_hbeta_log10"].isna()
df.loc[missing_obj, "target_hbeta_log10"] = df.loc[missing_obj, ID_COL].map(computed_obj_mean)

# Update source flag
df.loc[df["target_hbeta_source"].eq("missing") & df["target_hbeta_log10"].notna(), "target_hbeta_source"] = "computed_obj_mean"

# ----------------------------
# 3) Now drop ONLY rows whose object still has no Hβ target (website missing AND cannot compute)
# ----------------------------
before = df.shape[0]
df_final = df.dropna(subset=["target_hbeta_log10"]).copy()
dropped = before - df_final.shape[0]

print("\nAfter filling missing targets:")
print("  Rows before:", before)
print("  Rows after :", df_final.shape[0])
print("  Dropped rows (no website label + no computable Hβ rows for that object):", dropped)

print("\nObject-level coverage:")
print("  Unique objects total:", df[ID_COL].nunique())
print("  Unique objects kept :", df_final[ID_COL].nunique())

print("\nTarget source breakdown (rows):")
print(df_final["target_hbeta_source"].value_counts(dropna=False))

# ----------------------------
# 4) Save with comma delimiter
# ----------------------------
OUT_PATH = Path("agnmass_dump/reverb_tables/qc_finalized_data.csv")
df_final.to_csv(OUT_PATH, index=False, sep=",", encoding="utf-8-sig")
print("\nSaved:", OUT_PATH)


Loaded: agnmass_dump/reverb_tables/qc_project_cleaned_data.csv shape: (299, 77)

After filling missing targets:
  Rows before: 299
  Rows after : 261
  Dropped rows (no website label + no computable Hβ rows for that object): 38

Object-level coverage:
  Unique objects total: 86
  Unique objects kept : 67

Target source breakdown (rows):
target_hbeta_source
website    261
Name: count, dtype: int64

Saved: agnmass_dump/reverb_tables/qc_finalized_data.csv
